In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import os
import glob
from collections import deque, defaultdict
import pandas as pd
import numpy as np

<IPython.core.display.Javascript object>

In [3]:
import warnings

# Suppress specific FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn._oldcore")


<IPython.core.display.Javascript object>

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

<IPython.core.display.Javascript object>

In [5]:
from sklearn.preprocessing import StandardScaler

<IPython.core.display.Javascript object>

In [6]:
def read_csv_files_grouped(csv_files, plant, header=[0, 1]):
    results = []
    for filepath in csv_files:
        df = pd.read_csv(filepath, header=header, low_memory=False)
        df["plant"] = plant  # Add the 'plant' column
        results.append(df)
    df = pd.concat(results, ignore_index=True)
    return df

<IPython.core.display.Javascript object>

In [7]:
def read_csv_files_full(csv_files, plant):
    results = []
    for filepath in csv_files:
        df = pd.read_csv(filepath, low_memory=False)
        df["plant"] = plant  # Add the 'plant' column
        results.append(df)
    df = pd.concat(results, ignore_index=True)
    return df

<IPython.core.display.Javascript object>

### Renaming the Columns

In [8]:
def preprocess_rename_columns(df):
    df.columns = pd.MultiIndex.from_tuples(
        list(
            {
                col: (col[0], "") if "Unnamed" in col[1] else col for col in df.columns
            }.values()
        )
    )
    return df

<IPython.core.display.Javascript object>

### Changing the order_of columns Columns

In [9]:
def preprocess_change_columns_order(df, column, pos):
    # Get the list of columns
    cols = df.columns.tolist()
    # Remove the specified column
    cols.remove(column)
    # Insert the column at the desired position
    cols.insert(pos, column)
    # Reorder the DataFrame columns
    return df[cols]

<IPython.core.display.Javascript object>

In [10]:
def read_csv_files_path(csv_files_path_dict, path, plant):
    csv_files = glob.glob(os.path.join(path, "*.csv"))
    csv_files_path_dict[plant] = csv_files
    return csv_files_path_dict

<IPython.core.display.Javascript object>

# Reading The files

In [11]:
csv_files_path_grouped = dict()
csv_files_path_full = dict()

<IPython.core.display.Javascript object>

### 204

In [12]:
plants_204 = ["ab"]

for plant in plants_204:
    path_grouped = f"/home/peressim/projects/ccs28-ml-modelling/reports/results/local_models/204/{plant}/grouped/"
    path_full = f"/home/peressim/projects/ccs28-ml-modelling/reports/results/local_models/204/{plant}/full/"
    csv_files_path_full = read_csv_files_path(csv_files_path_full, path_full, plant)
    csv_files_path_grouped = read_csv_files_path(
        csv_files_path_grouped, path_grouped, plant
    )

<IPython.core.display.Javascript object>

### Reading all data

#### Full

In [13]:
dfs_plant = dict()
column = ("plant", "")
pos = 2

for plant, csv_files in csv_files_path_full.items():
    df = read_csv_files_full(csv_files, plant)
    df["plant"] = plant
    dfs_plant[plant] = df

<IPython.core.display.Javascript object>

In [14]:
df_full = pd.concat(dfs_plant.values(), ignore_index=True)

<IPython.core.display.Javascript object>

In [15]:
df_full.shape

(8648, 23)

<IPython.core.display.Javascript object>

In [16]:
filter_df = df_full[df_full["R2 Test"] < 0].reset_index(drop=True)

filter_df["CP"] = (
    filter_df["Company"].astype(str) + "_" + filter_df["plant"].astype(str)
)

filter_df["CP"].unique()

print(filter_df.shape)

(258, 24)


<IPython.core.display.Javascript object>

In [17]:
patterns = [
    "Chemical + Mineralogical + Feature Engineering",
    "Chemical + Mineralogical + CS7 + One-Hot",
    "Chemical + Mineralogical + Physical + One-Hot",
    "Chemical + Mineralogical + CS3 + One-Hot",
    "Chemical + Mineralogical + CS3 + CS7 + One-Hot",
    "Chemical + Mineralogical + One-Hot",
    "Chemical + Mineralogical + CS1 + CS3 + CS7 + One-Hot",
    "Chemical + Mineralogical + CS1 + CS3 + One-Hot",
    "Chemical + Mineralogical + CS1 + One-Hot",
    "Chemical + Mineralogical + CS2 + One-Hot",
    "Chemical + Feature Engineering",
]

FEATURES_TO_REPLACE_1 = {
    "Chemical + Mineralogical + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1 + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1 + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS2": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS7": "Chemical + Mineralogical + Early CS",
}

FEATURES_TO_REPLACE_2 = {
    "Chemical": "Chemical + Mineralogical",
    "Chemical + Properties CS Less": "Chemical + Mineralogical + Physical - Early CS",
    "Chemical + Mineralogical + Properties CS Less": "Chemical + Mineralogical + Physical - Early CS",
    "Chemical + Physical": "Chemical + Mineralogical + Physical",
}

df_full = df_full[~df_full["Features"].apply(lambda x: x in patterns)].reset_index(
    drop=True
)

df_full = df_full[~df_full["Cross Validation"].eq("Out of time Split")].reset_index(
    drop=True
)

# removing afterwards
df_full = df_full[~df_full["Cross Validation"].eq("Repeated KFold")].reset_index(
    drop=True
)

df_full["Features_bkp"] = df_full["Features"].copy()
df_full["Features"] = df_full["Features"].replace(FEATURES_TO_REPLACE_1)

df_full["Features"] = df_full["Features"].replace(FEATURES_TO_REPLACE_2)


# removing afterwards
df_full = df_full[
    ~df_full["Features"].eq("Chemical + Mineralogical + Early CS")
].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [18]:
df_full["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [19]:
df_full["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS',
       'Chemical + Mineralogical + Physical', 'Chemical + Mineralogical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [20]:
df_full["Features_bkp"].unique()

array(['Chemical + Properties CS Less', 'Chemical + Physical', 'Chemical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [21]:
df_full.shape

(1560, 24)

<IPython.core.display.Javascript object>

In [22]:
df_full[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
RMSE Test,1560.0,2.81,1.76,0.99,1.71,2.26,3.25,12.57
MAE Test,1560.0,2.18,1.27,0.73,1.37,1.80,2.61,10.23
MAPE Test,1560.0,0.05,0.03,0.02,0.03,0.04,0.06,0.24
R2 Test,1560.0,0.62,0.95,-7.24,0.74,0.87,0.92,0.98


<IPython.core.display.Javascript object>

In [23]:
df_full[df_full["R2 Test"] < 0][["Company", "plant"]]["plant"].nunique()

1

<IPython.core.display.Javascript object>

In [24]:
df_full[df_full["R2 Test"].eq(df_full["R2 Test"].min())]

,Category,Company,Plant,Features,Data Shape,Timesteps,Model,Model Params,Scaler,Scaler Params,...,RMSE Train,MAE Train,MAPE Train,R2 Train,RMSE Test,MAE Test,MAPE Test,R2 Test,plant,Features_bkp
519,Local Model,204,AB,Chemical + Mineralogical + Physical - Early CS,"(2313, 10)",7.0,BidirectionalLSTM,NaN,Standard Scaler,NaN,...,1.010253,0.811362,0.01849,0.967772,12.57345,9.552971,0.234658,-7.242704,ab,Chemical + Properties CS Less
524,Local Model,204,AB,Chemical + Mineralogical + Physical - Early CS,"(2313, 10)",7.0,BidirectionalLSTM,NaN,Standard Scaler,NaN,...,1.010253,0.811362,0.01849,0.967772,12.57345,9.552971,0.234658,-7.242704,ab,Chemical + Properties CS Less
529,Local Model,204,AB,Chemical + Mineralogical + Physical - Early CS,"(2313, 10)",7.0,BidirectionalLSTM,NaN,Standard Scaler,NaN,...,1.010253,0.811362,0.01849,0.967772,12.57345,9.552971,0.234658,-7.242704,ab,Chemical + Properties CS Less


<IPython.core.display.Javascript object>

In [25]:
df_full[(df_full["R2 Test"].eq(df_full["R2 Test"].min()))]["Cross Validation"]

519    Blocking Time Series Split
524    Blocking Time Series Split
529    Blocking Time Series Split
Name: Cross Validation, dtype: object

<IPython.core.display.Javascript object>

In [26]:
filter_df = df_full[df_full["R2 Test"] < 0].reset_index(drop=True)

print(filter_df.shape)

(99, 24)


<IPython.core.display.Javascript object>

In [27]:
df_full.shape

(1560, 24)

<IPython.core.display.Javascript object>

In [28]:
df_full["plant"].unique()

array(['ab'], dtype=object)

<IPython.core.display.Javascript object>

In [29]:
df_full_204 = df_full[df_full["plant"].isin(["partner_i", "partner_ii", "partner_iv"])].reset_index(
    drop=True
)
df_full_204 = df_full[~df_full["plant"].isin(["partner_i", "partner_ii", "partner_iv"])].reset_index(
    drop=True
)

<IPython.core.display.Javascript object>

In [30]:
df_full.shape

(1560, 24)

<IPython.core.display.Javascript object>

In [31]:
df_full_204.shape

(1560, 24)

<IPython.core.display.Javascript object>

In [32]:
df_full_204.shape

(1560, 24)

<IPython.core.display.Javascript object>

In [33]:
df_full_204[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
RMSE Test,1560.0,2.81,1.76,0.99,1.71,2.26,3.25,12.57
MAE Test,1560.0,2.18,1.27,0.73,1.37,1.80,2.61,10.23
MAPE Test,1560.0,0.05,0.03,0.02,0.03,0.04,0.06,0.24
R2 Test,1560.0,0.62,0.95,-7.24,0.74,0.87,0.92,0.98


<IPython.core.display.Javascript object>

In [34]:
df_full_204[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
RMSE Test,1560.0,2.81,1.76,0.99,1.71,2.26,3.25,12.57
MAE Test,1560.0,2.18,1.27,0.73,1.37,1.80,2.61,10.23
MAPE Test,1560.0,0.05,0.03,0.02,0.03,0.04,0.06,0.24
R2 Test,1560.0,0.62,0.95,-7.24,0.74,0.87,0.92,0.98


<IPython.core.display.Javascript object>

# Analysis (full results)

In [35]:
df_full_204 = df_full[~df_full["plant"].isin(["partner_i", "partner_ii", "partner_iv"])].reset_index(
    drop=True
)

<IPython.core.display.Javascript object>

## 204

In [36]:
df_full_204.shape

(1560, 24)

<IPython.core.display.Javascript object>

In [37]:
df_full_204["Company"].unique()

array([204])

<IPython.core.display.Javascript object>

### Variable Grouping: CM

In [38]:
df_full_204["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS',
       'Chemical + Mineralogical + Physical', 'Chemical + Mineralogical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [39]:
df_results_full_cm = (
    df_full_204[df_full_204["Features"].eq("Chemical + Mineralogical")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [40]:
df_results_full_cm["Features"].unique()

array(['Chemical + Mineralogical'], dtype=object)

<IPython.core.display.Javascript object>

In [41]:
df_results_full_cm.shape

(520, 24)

<IPython.core.display.Javascript object>

#### K-Fold

In [42]:
df_results_full_cm["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [43]:
df_results_full_cm_kf = (
    df_results_full_cm[df_results_full_cm["Cross Validation"].eq("Repeated KFold")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [44]:
df_results_full_cm_kf["Cross Validation"].unique()

array([], dtype=object)

<IPython.core.display.Javascript object>

In [45]:
df_results_full_cm_kf.shape

(0, 24)

<IPython.core.display.Javascript object>

In [46]:
df_results_full_cm_kf.groupby(["Company", "Model", "Timesteps"], dropna=False)[
    "RMSE Test"
].mean().shape

(0,)

<IPython.core.display.Javascript object>

### Blocking time series

In [47]:
df_results_full_cm["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [48]:
df_results_full_cm_btss = (
    df_results_full_cm[
        df_results_full_cm["Cross Validation"].eq("Blocking Time Series Split")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [49]:
df_results_full_cm_btss["Cross Validation"].unique()

array(['Blocking Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [50]:
df_results_full_cm_btss.shape

(260, 24)

<IPython.core.display.Javascript object>

In [51]:
df_results_full_cm_btss.groupby(
    ["Company", "Plant", "Model", "Timesteps"], dropna=False
)["RMSE Test"].mean().shape

(20,)

<IPython.core.display.Javascript object>

### Time Series Split

In [52]:
df_results_full_cm["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [53]:
df_results_full_cm_tss = (
    df_results_full_cm[df_results_full_cm["Cross Validation"].eq("Time Series Split")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [54]:
df_results_full_cm_tss["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [55]:
df_results_full_cm_tss.shape

(260, 24)

<IPython.core.display.Javascript object>

In [56]:
df_results_full_cm_tss.groupby(
    ["Company", "Plant", "Model", "Timesteps"], dropna=False
)["RMSE Test"].mean().shape

(20,)

<IPython.core.display.Javascript object>

### Variable Grouping: CM-P

In [57]:
df_full_204["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS',
       'Chemical + Mineralogical + Physical', 'Chemical + Mineralogical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [58]:
df_results_full_cm_p = (
    df_full_204[
        df_full_204["Features"].eq("Chemical + Mineralogical + Physical - Early CS")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [59]:
df_results_full_cm_p["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [60]:
df_results_full_cm_p.shape

(520, 24)

<IPython.core.display.Javascript object>

### K-Fold

In [61]:
# df_results_full_cm_p["Cross Validation"].unique()

<IPython.core.display.Javascript object>

In [62]:
# df_results_full_cm_p_kf = (
#     df_results_full_cm_p[df_results_full_cm_p["Cross Validation"].eq("Repeated KFold")]
#     .copy()
#     .reset_index(drop=True)
# )

<IPython.core.display.Javascript object>

In [63]:
# df_results_full_cm_p_kf["Cross Validation"].unique()

<IPython.core.display.Javascript object>

In [64]:
# df_results_full_cm_p_kf.shape

<IPython.core.display.Javascript object>

In [65]:
# df_results_full_cm_kf.groupby(["Company", "Model", "Timesteps"], dropna=False)[
#     "RMSE Test"
# ].mean().shape

<IPython.core.display.Javascript object>

### Blocking time series

In [66]:
df_results_full_cm_p["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [67]:
df_results_full_cm_p_btss = (
    df_results_full_cm_p[
        df_results_full_cm_p["Cross Validation"].eq("Blocking Time Series Split")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [68]:
df_results_full_cm_p_btss["Cross Validation"].unique()

array(['Blocking Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [69]:
df_results_full_cm_p_btss.shape

(260, 24)

<IPython.core.display.Javascript object>

In [70]:
df_results_full_cm_p_btss.shape, df_results_full_cm_btss.shape

((260, 24), (260, 24))

<IPython.core.display.Javascript object>

In [71]:
df_results_full_cm_p_btss.groupby(
    ["Company", "Cross Validation", "Plant", "Model", "Timesteps"], dropna=False
)["RMSE Test"].mean().shape

(20,)

<IPython.core.display.Javascript object>

In [72]:
df_results_full_cm_btss.shape, df_results_full_cm_p_btss.shape

df1 = df_results_full_cm_btss
df2 = df_results_full_cm_p_btss

grouping_columns = ["Company", "Plant", "Model", "Timesteps"]

<IPython.core.display.Javascript object>

In [73]:
# Group by and get sizes for both datasets
group_sizes_df1 = df1.groupby(grouping_columns, dropna=False).size()
group_sizes_df2 = df2.groupby(grouping_columns, dropna=False).size()

# Compare group sizes directly
group_size_diff = group_sizes_df1.compare(group_sizes_df2)
print("Differences in group sizes:")
print(group_size_diff)

Differences in group sizes:
Empty DataFrame
Columns: [self, other]
Index: []


<IPython.core.display.Javascript object>

In [74]:
# Group by and get sizes for both datasets
group_sizes_df1 = df1.groupby(grouping_columns, dropna=False).size()
group_sizes_df2 = df2.groupby(grouping_columns, dropna=False).size()

# Compare group sizes directly
group_size_diff = group_sizes_df2.compare(group_sizes_df1)
print("Differences in group sizes:")
print(group_size_diff)

Differences in group sizes:
Empty DataFrame
Columns: [self, other]
Index: []


<IPython.core.display.Javascript object>

### Time Series Split

In [75]:
df_results_full_cm_p["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [76]:
df_results_full_cm_p_tss = (
    df_results_full_cm_p[
        df_results_full_cm_p["Cross Validation"].eq("Time Series Split")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [77]:
df_results_full_cm_p_tss["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [78]:
df_results_full_cm_p_tss.shape

(260, 24)

<IPython.core.display.Javascript object>

In [79]:
df_results_full_cm_p_tss.groupby(
    ["Company", "Model", "Plant", "Timesteps"], dropna=False
)["RMSE Test"].mean().shape

(20,)

<IPython.core.display.Javascript object>

### Variable Grouping: CM-P-CS

In [80]:
df_full_204["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS',
       'Chemical + Mineralogical + Physical', 'Chemical + Mineralogical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [81]:
df_results_full_cm_p_cs = (
    df_full_204[df_full_204["Features"].eq("Chemical + Mineralogical + Physical")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [82]:
df_results_full_cm_p_cs["Features"].unique()

array(['Chemical + Mineralogical + Physical'], dtype=object)

<IPython.core.display.Javascript object>

In [83]:
df_results_full_cm_p_cs.shape

(520, 24)

<IPython.core.display.Javascript object>

### K-Fold

In [84]:
df_results_full_cm_p_cs["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [85]:
df_results_full_cm_p_cs_kf = (
    df_results_full_cm_p_cs[
        df_results_full_cm_p_cs["Cross Validation"].eq("Repeated KFold")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [86]:
df_results_full_cm_p_cs_kf["Cross Validation"].unique()

array([], dtype=object)

<IPython.core.display.Javascript object>

In [87]:
df_results_full_cm_p_cs_kf.shape

(0, 24)

<IPython.core.display.Javascript object>

In [88]:
df_results_full_cm_p_cs_kf.groupby(["Company", "Model", "Timesteps"], dropna=False)[
    "RMSE Test"
].mean().shape

(0,)

<IPython.core.display.Javascript object>

### Blocking time series

In [89]:
df_results_full_cm_p_cs["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [90]:
df_results_full_cm_p_cs_btss = (
    df_results_full_cm_p_cs[
        df_results_full_cm_p_cs["Cross Validation"].eq("Blocking Time Series Split")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [91]:
df_results_full_cm_p_cs_btss["Cross Validation"].unique()

array(['Blocking Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [92]:
df_results_full_cm_p_cs_btss.shape

(260, 24)

<IPython.core.display.Javascript object>

In [93]:
df_results_full_cm_p_cs_btss.groupby(
    ["Company", "Plant", "Model", "Timesteps"], dropna=False
)["RMSE Test"].mean().shape

(20,)

<IPython.core.display.Javascript object>

### Time Series Split

In [94]:
df_results_full_cm_p_cs["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [95]:
df_results_full_cm_p_cs_tss = (
    df_results_full_cm_p_cs[
        df_results_full_cm_p_cs["Cross Validation"].eq("Time Series Split")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [96]:
df_results_full_cm_p_cs_tss["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [97]:
df_results_full_cm_p_cs_tss.shape

(260, 24)

<IPython.core.display.Javascript object>

In [98]:
df_results_full_cm_p_cs_tss.groupby(
    ["Company", "Plant", "Model", "Timesteps"], dropna=False
)["RMSE Test"].mean().shape

(20,)

<IPython.core.display.Javascript object>

# Grouped Dataset

In [99]:
dfs_plant = dict()
column = ("plant", "")
pos = 2

for plant, csv_files in csv_files_path_grouped.items():
    df = read_csv_files_grouped(csv_files, plant)
    df["plant"] = plant
    df = preprocess_rename_columns(df)
    df = preprocess_change_columns_order(df, column, pos)
    dfs_plant[plant] = df

<IPython.core.display.Javascript object>

In [100]:
df = pd.concat(dfs_plant)
df_copy = df.copy()
df_copy = (
    df_copy.reset_index(level=0)
    .rename({"level_0": "Plant"}, axis=1)
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [101]:
df_copy.shape

(794, 26)

<IPython.core.display.Javascript object>

In [102]:
df_full_filter = df_full[
    df_full["Cross Validation"].eq("Time Series Split")
].reset_index(drop=True)

df_full_groupped = df_full_filter.groupby(
    [
        "Category",
        "Company",
        "Features",
        "Data Shape",
        "Timesteps",
        "Model",
        "Cross Validation",
        "Cross Validation Params",
        "plant",
        "Plant",
    ],
    dropna=False,
    as_index=False,
).last()

<IPython.core.display.Javascript object>

In [103]:
df_copy = df_full_groupped

<IPython.core.display.Javascript object>

## Preprocessing steps

In [104]:
patterns = [
    "Chemical + Mineralogical + Feature Engineering",
    "Chemical + Mineralogical + CS7 + One-Hot",
    "Chemical + Mineralogical + Physical + One-Hot",
    "Chemical + Mineralogical + CS3 + One-Hot",
    "Chemical + Mineralogical + CS3 + CS7 + One-Hot",
    "Chemical + Mineralogical + One-Hot",
    "Chemical + Mineralogical + CS1 + CS3 + CS7 + One-Hot",
    "Chemical + Mineralogical + CS1 + CS3 + One-Hot",
    "Chemical + Mineralogical + CS1 + One-Hot",
    "Chemical + Mineralogical + CS2 + One-Hot",
    "Chemical + Feature Engineering",
]

replace_dict = {
    "Chemical + Mineralogical + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1 + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1 + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS2": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS7": "Chemical + Mineralogical + Early CS",
}

replace_dict_2 = {
    "Chemical": "Chemical + Mineralogical",
    "Chemical + Properties CS Less": "Chemical + Mineralogical + Physical - Early CS",
    "Chemical + Mineralogical + Properties CS Less": "Chemical + Mineralogical + Physical - Early CS",
    "Chemical + Physical": "Chemical + Mineralogical + Physical",
}

df_copy = df_copy[~df_copy["Cross Validation"].eq("Out of time Split")].reset_index(
    drop=True
)

# Removing afterwards
df_copy = df_copy[~df_copy["Cross Validation"].eq("Repeated KFold")].reset_index(
    drop=True
)

df_copy = df_copy[~df_copy["Features"].apply(lambda x: x in patterns)].reset_index(
    drop=True
)
df_copy["Features_bkp"] = df_copy["Features"].copy()
df_copy["Features"] = df_copy["Features"].replace(replace_dict)

df_copy["Features"] = df_copy["Features"].replace(replace_dict_2)

# Removing afterwards
df_copy = df_copy[
    ~df_copy["Features"].eq("Chemical + Mineralogical + Early CS")
].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [105]:
df_copy["MAPE Train"] = df_copy["MAPE Train"] * 100
df_copy["MAPE Test"] = df_copy["MAPE Test"] * 100

df_copy["Model_bkp"] = df_copy["Model"].copy()
df_copy["Model_bkp_2"] = df_copy["Model"] + df_copy["Timesteps"].apply(
    lambda x: "" if pd.isna(x) else "_" + str(x)
)
df_copy["Model"] = df_copy["Model"].replace(
    {
        "MLP": "Neural Networks",
        "LSTM": "Neural Networks",
        "GRU": "Neural Networks",
        "BidirectionalLSTM": "Neural Networks",
        "BidirectionalGRU": "Neural Networks",
        "Transformer": "Neural Networks",
        "Decision Tree": "Trees",
        "Random Forest": "Trees",
        "XGBoost": "Trees",
    }
)

<IPython.core.display.Javascript object>

In [106]:
df_copy["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [107]:
df_copy["Features"].unique()

array(['Chemical + Mineralogical', 'Chemical + Mineralogical + Physical',
       'Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [108]:
df_copy.shape

(60, 26)

<IPython.core.display.Javascript object>

In [109]:
df_copy["plant"].unique()

array(['ab'], dtype=object)

<IPython.core.display.Javascript object>

In [110]:
df_full.shape

(1560, 24)

<IPython.core.display.Javascript object>

In [111]:
cols_groupby = [
    # "Category",
    # "Company",
    "plant",
    # "Data Shape",
    "Timesteps",
    "Features_bkp",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

df_results_to_save = (
    df_full.groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

<IPython.core.display.Javascript object>

In [112]:
df_results_to_save.shape

(120, 22)

<IPython.core.display.Javascript object>

In [113]:
df_copy.shape

(60, 26)

<IPython.core.display.Javascript object>

In [114]:
df_copy_grouped = df_copy.copy()

<IPython.core.display.Javascript object>

In [115]:
df_copy_grouped.shape

(60, 26)

<IPython.core.display.Javascript object>

# Topological Analysis - NEW

## Definitions

In [116]:
# Function to compare two models strictly based on means only
def dominates(model1, model2):
    """Return True if model1 strictly dominates model2, False otherwise."""
    # Check if model1 is at least as good as model2 in all metrics
    at_least_as_good = (
        model1["RMSE_mean"] <= model2["RMSE_mean"]
        and model1["MAE_mean"] <= model2["MAE_mean"]
        and model1["MAPE_mean"] <= model2["MAPE_mean"]
        and model1["R2_mean"] >= model2["R2_mean"]
    )

    # Check if model1 is strictly better in at least one metric
    strictly_better_in_one = (
        model1["RMSE_mean"] < model2["RMSE_mean"]
        or model1["MAE_mean"] < model2["MAE_mean"]
        or model1["MAPE_mean"] < model2["MAPE_mean"]
        or model1["R2_mean"] > model2["R2_mean"]
    )

    # Model1 strictly dominates Model2 if it is at least as good in all metrics and
    # strictly better in at least one metric
    return at_least_as_good and strictly_better_in_one

<IPython.core.display.Javascript object>

In [117]:
def get_dominance_matrix_and_graph(df):
    n = len(df)
    dominance_matrix = np.zeros((n, n), dtype=bool)

    for i in range(n):
        for j in range(n):
            if i != j:
                dominance_matrix[i, j] = dominates(df.iloc[i], df.iloc[j])

    # Create the dominance graph
    dominance_graph = {i: [] for i in range(n)}
    for i in range(n):
        for j in range(n):
            if dominance_matrix[i, j]:
                dominance_graph[i].append(j)

    return dominance_matrix, dominance_graph

<IPython.core.display.Javascript object>

In [118]:
# Topological Sorting
def topological_sort(graph):
    """Perform topological sorting on the given graph."""
    in_degree = {u: 0 for u in graph}
    for u in graph:
        for v in graph[u]:
            in_degree[v] += 1

    queue = deque([u for u in graph if in_degree[u] == 0])
    topo_order = []

    while queue:
        u = queue.popleft()
        topo_order.append(u)

        for v in graph[u]:
            in_degree[v] -= 1
            if in_degree[v] == 0:
                queue.append(v)

    return topo_order

<IPython.core.display.Javascript object>

In [119]:
# Function to find the top models in each group
def find_top_models(group):
    max_net_dominance = group["Net_Dominance"].max()
    top_models = group[group["Net_Dominance"] == max_net_dominance]
    return top_models

<IPython.core.display.Javascript object>

### Dominance Analysis and SCPM

In [120]:
def compute_scpm(df):
    scaler = StandardScaler()
    standardized_metrics = scaler.fit_transform(
        df[["RMSE_mean", "MAE_mean", "MAPE_mean", "R2_mean"]]
    )

    # Creating a new dataframe with standardized values
    standardized_df = pd.DataFrame(
        standardized_metrics,
        columns=["RMSE_mean", "MAE_mean", "MAPE_mean", "R2_mean"],
    )

    # Summing all metrics and subtracting R2_mean
    standardized_df["Result"] = (
        standardized_df[["RMSE_mean", "MAE_mean", "MAPE_mean"]].sum(axis=1)
        - standardized_df["R2_mean"]
    )

    df["SCPM"] = standardized_df["Result"]
    return df

<IPython.core.display.Javascript object>

In [121]:
def make_dominance_analysis(df):
    # Compute dominance matrix and graph
    dominance_matrix, dominance_graph = get_dominance_matrix_and_graph(df)

    # Get the topological order
    topo_order = topological_sort(dominance_graph)
    df = df.copy()
    n = len(topo_order)

    # Identify Dominant Models
    dominant_models = [
        df["Model_bkp_2"].iloc[i]
        for i in range(len(df))
        if not np.any(dominance_matrix[:, i])
    ]

    # Identify Non-Dominant Models
    non_dominant_models = [
        df["Model_bkp_2"].iloc[i]
        for i in range(len(df))
        if np.any(dominance_matrix[:, i])
    ]

    # Identify Models that are both dominated and dominate at least one
    intermediate_models = [
        df["Model_bkp_2"].iloc[i]
        for i in range(len(df))
        if np.any(dominance_matrix[:, i]) and np.any(dominance_matrix[i, :])
    ]

    # Add count of times each model is dominated by others
    df["Dominated_Count"] = np.sum(dominance_matrix, axis=0)

    # Classify models
    df["Classification"] = "intermediate_model"  # default to intermediate_model
    df.loc[
        ~np.any(dominance_matrix, axis=1) & np.any(dominance_matrix, axis=0),
        "Classification",
    ] = "non_dominant_model"
    df.loc[~np.any(dominance_matrix, axis=0), "Classification"] = "dominant_model"

    # Map the topological order to the model names and get dominance counts
    sorted_models = [df["Model"].iloc[i] for i in topo_order]

    df_sorted = df.iloc[topo_order].copy()
    df_sorted["Dominates_Count"] = dominance_matrix.sum(axis=1)[topo_order]
    df_sorted_topo = df_sorted.copy()

    df_sorted = df_sorted.sort_values(by="Dominates_Count", ascending=False)
    df_sorted_count = df_sorted.copy()

    return {
        "dominance_matrix": dominance_matrix,
        "dominance_graph": dominance_graph,
        "df_sorted_topo": df_sorted_topo,
        "df_sorted_count": df_sorted_count,
    }

<IPython.core.display.Javascript object>

# Analysis

## 204

In [122]:
# Filter only 204 project
df_204_grouped = df_copy_grouped[
    ~df_copy_grouped["plant"].isin(["partner_i", "partner_ii", "partner_iv"])
]

<IPython.core.display.Javascript object>

In [123]:
df_results = df_204_grouped[
    [
        ("Company"),
        ("plant"),
        ("Cross Validation"),
        ("Features"),
        ("Features_bkp"),
        ("Model"),
        ("Model_bkp"),
        ("Model_bkp_2"),
        ("Timesteps"),
        ("RMSE Test"),
        ("MAE Test"),
        ("MAPE Test"),
        ("R2 Test"),
    ]
].copy()
df_results = df_results.reset_index(drop=True)

# Rename columns
new_column_names = [
    "Company",
    "Plant",
    "Cross Validation",
    "Features",
    "Features_bkp",
    "Model",
    "Model_bkp",
    "Model_bkp_2",
    "Timesteps",
    "RMSE_mean",
    "MAE_mean",
    "MAPE_mean",
    "R2_mean",
]

df_results.columns = new_column_names

<IPython.core.display.Javascript object>

In [124]:
df_results.shape

(60, 13)

<IPython.core.display.Javascript object>

In [125]:
df_results["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

## Variable Grouping: CM

In [126]:
df_results["Features"].unique()

array(['Chemical + Mineralogical', 'Chemical + Mineralogical + Physical',
       'Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [127]:
df_results_cm = (
    df_results[df_results["Features"].eq("Chemical + Mineralogical")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [128]:
df_results_cm["Features"].unique()

array(['Chemical + Mineralogical'], dtype=object)

<IPython.core.display.Javascript object>

In [129]:
df_results_cm.shape

(20, 13)

<IPython.core.display.Javascript object>

### Time Series Split

In [130]:
df_results_cm["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [131]:
df_results_cm_tss = (
    df_results_cm[df_results_cm["Cross Validation"].eq("Time Series Split")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [132]:
df_results_cm_tss["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [133]:
df_results_cm_tss.shape

(20, 13)

<IPython.core.display.Javascript object>

###### SCPM computation

In [134]:
df_results_cm_tss = compute_scpm(df_results_cm_tss)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [135]:
# Per project
dominance_dict = make_dominance_analysis(df_results_cm_tss)
dominance_matrix_cm_tss = dominance_dict["dominance_matrix"]
dominance_graph_cm_tss = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [136]:
dominance_matrix_cm_tss.shape, len(dominance_graph_cm_tss)

((20, 20), 20)

<IPython.core.display.Javascript object>

In [137]:
df_sorted_topo["Classification"].value_counts()

Classification
intermediate_model    17
dominant_model         2
non_dominant_model     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [138]:
df_sorted_topo.to_csv(
    "../../../../reports/results/local_models/ecics/204_ab_dominance_analysis_cm_tss_last_fold.csv",
    index=False,
)

<IPython.core.display.Javascript object>

In [139]:
df_sorted_topo.head(5)

,Company,Plant,Cross Validation,Features,Features_bkp,Model,Model_bkp,Model_bkp_2,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count
4,204,ab,Time Series Split,Chemical + Mineralogical,Chemical + Mineralogical,Neural Networks,Transformer,Transformer_1.0,1.0,1.494908,1.086554,2.539057,0.919738,-5.010979,0,dominant_model,18
13,204,ab,Time Series Split,Chemical + Mineralogical,Chemical + Mineralogical,Neural Networks,LSTM,LSTM_14.0,14.0,1.493346,1.127769,2.674217,0.919466,-4.516973,0,dominant_model,17
0,204,ab,Time Series Split,Chemical + Mineralogical,Chemical + Mineralogical,Neural Networks,BidirectionalGRU,BidirectionalGRU_1.0,1.0,1.509646,1.118925,2.641868,0.918148,-4.519308,1,intermediate_model,17
1,204,ab,Time Series Split,Chemical + Mineralogical,Chemical + Mineralogical,Neural Networks,BidirectionalLSTM,BidirectionalLSTM_1.0,1.0,1.536849,1.188111,2.774209,0.915171,-3.692015,3,intermediate_model,15
19,204,ab,Time Series Split,Chemical + Mineralogical,Chemical + Mineralogical,Trees,XGBoost,XGBoost,NaN,1.682781,1.153445,2.795127,0.898296,-2.661031,3,intermediate_model,11


<IPython.core.display.Javascript object>

In [140]:
df_sorted_topo_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "RMSE_mean",
        "MAE_mean",
        "MAPE_mean",
        "R2_mean",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
].round(2)

<IPython.core.display.Javascript object>

###### Dominant Models

In [141]:
# Top 5
df_sorted_topo_cols.iloc[0:5].sort_values(
    by=["Dominates_Count", "SCPM"], ascending=[False, True]
).round(2)

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
4,Transformer_1.0,1.49,1.09,2.54,0.92,-5.01,0,dominant_model,18,ab
13,LSTM_14.0,1.49,1.13,2.67,0.92,-4.52,0,dominant_model,17,ab
0,BidirectionalGRU_1.0,1.51,1.12,2.64,0.92,-4.52,1,intermediate_model,17,ab
1,BidirectionalLSTM_1.0,1.54,1.19,2.77,0.92,-3.69,3,intermediate_model,15,ab
19,XGBoost,1.68,1.15,2.80,0.90,-2.66,3,intermediate_model,11,ab


<IPython.core.display.Javascript object>

In [142]:
df_sorted_topo_cols[df_sorted_topo_cols["Classification"].eq("dominant_model")].shape

(2, 10)

<IPython.core.display.Javascript object>

###### Intermediate models

In [143]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:4].round(2)

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
0,BidirectionalGRU_1.0,1.51,1.12,2.64,0.92,-4.52,1,intermediate_model,17,ab
1,BidirectionalLSTM_1.0,1.54,1.19,2.77,0.92,-3.69,3,intermediate_model,15,ab
3,LSTM_1.0,1.59,1.26,2.93,0.91,-2.54,4,intermediate_model,13,ab
2,GRU_1.0,1.60,1.26,2.93,0.91,-2.51,4,intermediate_model,13,ab


<IPython.core.display.Javascript object>

In [144]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].shape

(17, 10)

<IPython.core.display.Javascript object>

###### non-dominant models

In [145]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:3]

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
16,Linear Regression,2.31,1.84,4.15,0.81,9.03,19,non_dominant_model,0,ab


<IPython.core.display.Javascript object>

In [146]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].shape

(1, 10)

<IPython.core.display.Javascript object>

###### Statistics per model type

In [147]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "RMSE_mean",
        "MAE_mean",
        "MAPE_mean",
        "R2_mean",
        "SCPM",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [148]:
df_sorted_topo["Model"].value_counts()

Model
Neural Networks      16
Trees                 3
Linear Regression     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [149]:
df_sorted_topo_models_grouped = df_sorted_topo_models_cols.groupby("Model").describe().T

<IPython.core.display.Javascript object>

In [150]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [151]:
(
    df_sorted_topo_models_cols.groupby("Model")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model             Classification                
Linear Regression non_dominant_model       100.0
Neural Networks   intermediate_model        87.5
                  dominant_model            12.5
Trees             intermediate_model       100.0

<IPython.core.display.Javascript object>

In [152]:
df_sorted_topo_models_cols.groupby("Model").describe().round(2).T

Model                  Linear Regression  Neural Networks  Trees
Dominated_Count count                1.0            16.00   3.00
                mean                19.0             8.31   6.33
                std                  NaN             6.03   3.06
                min                 19.0             0.00   3.00
                25%                 19.0             3.75   5.00
                50%                 19.0             8.00   7.00
                75%                 19.0            13.25   8.00
                max                 19.0            17.00   9.00
Dominates_Count count                1.0            16.00   3.00
                mean                 0.0             9.19   8.00
                std                  NaN             5.82   3.00
                min                  0.0             1.00   5.00
                25%                  0.0             4.00   6.50
                50%                  0.0             8.50   8.00
                75%                  0.0            13.50   9.50
                max                  0.0            18.00  11.00

<IPython.core.display.Javascript object>

In [153]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [154]:
df_sorted_topo_models_cols.groupby("Model")["Classification"].describe()

,count,unique,top,freq
Model,,,,
Linear Regression,1,1,non_dominant_model,1
Neural Networks,16,2,intermediate_model,14
Trees,3,1,intermediate_model,3


<IPython.core.display.Javascript object>

In [155]:
df_sorted_topo_models_cols.groupby("Model")[["Classification"]].value_counts()

Model              Classification    
Linear Regression  non_dominant_model     1
Neural Networks    intermediate_model    14
                   dominant_model         2
Trees              intermediate_model     3
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [156]:
summary_stats = df_sorted_topo_models_cols.groupby("Model").agg(
    Total_Models=("Model", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [157]:
summary_stats.round(2)

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model,,,,,,
Linear Regression,1,0.00,0.0,19,0,0.00
Neural Networks,16,9.19,8.5,133,147,7.35
Trees,3,8.00,8.0,19,24,1.20


<IPython.core.display.Javascript object>

In [158]:
df_sorted_topo_models_grouped.T

RMSE_mean                                                    \
                      count      mean       std       min       25%       50%   
Model                                                                           
Linear Regression       1.0  2.306547       NaN  2.306547  2.306547  2.306547   
Neural Networks        16.0  1.745489  0.229360  1.493346  1.580263  1.690685   
Trees                   3.0  1.794617  0.098408  1.682781  1.757944  1.833108   

                                      MAE_mean            ... Dominated_Count  \
                        75%       max    count      mean  ...             75%   
Model                                                     ...                   
Linear Regression  2.306547  2.306547      1.0  1.842424  ...           19.00   
Neural Networks    1.869148  2.262693     16.0  1.353347  ...           13.25   
Trees              1.850535  1.867962      3.0  1.241931  ...            8.00   

                        Dominates_Count                                   \
                    max           count    mean       std  min  25%  50%   
Model                                                                      
Linear Regression  19.0             1.0  0.0000       NaN  0.0  0.0  0.0   
Neural Networks    17.0            16.0  9.1875  5.822013  1.0  4.0  8.5   
Trees               9.0             3.0  8.0000  3.000000  5.0  6.5  8.0   

                               
                    75%   max  
Model                          
Linear Regression   0.0   0.0  
Neural Networks    13.5  18.0  
Trees               9.5  11.0  

[3 rows x 56 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [159]:
cols = [
    # "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "RMSE_mean",
    "MAE_mean",
    "MAPE_mean",
    "R2_mean",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

#### SCPM Analysis by Project

In [160]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [161]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [162]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(5)

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
4,Transformer_1.0,1.49,1.09,2.54,0.92,-5.01,0,18,dominant_model,ab,True
0,BidirectionalGRU_1.0,1.51,1.12,2.64,0.92,-4.52,1,17,intermediate_model,ab,False
13,LSTM_14.0,1.49,1.13,2.67,0.92,-4.52,0,17,dominant_model,ab,False
1,BidirectionalLSTM_1.0,1.54,1.19,2.77,0.92,-3.69,3,15,intermediate_model,ab,True
19,XGBoost,1.68,1.15,2.80,0.90,-2.66,3,11,intermediate_model,ab,True


<IPython.core.display.Javascript object>

### Variable Grouping: CM-P

In [163]:
df_results["Features"].unique()

array(['Chemical + Mineralogical', 'Chemical + Mineralogical + Physical',
       'Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [164]:
df_results_cm_p = (
    df_results[
        df_results["Features"].eq("Chemical + Mineralogical + Physical - Early CS")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [165]:
df_results_cm_p["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [166]:
df_results_cm_p.shape

(20, 13)

<IPython.core.display.Javascript object>

### Time Series Split

In [167]:
df_results_cm_p["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [168]:
df_results_cm_p_tss = (
    df_results_cm_p[df_results_cm_p["Cross Validation"].eq("Time Series Split")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [169]:
df_results_cm_p_tss["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [170]:
df_results_cm_p_tss.shape

(20, 13)

<IPython.core.display.Javascript object>

###### SCPM computation

In [171]:
df_results_cm_p_tss = compute_scpm(df_results_cm_p_tss)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [172]:
# Per project
dominance_dict = make_dominance_analysis(df_results_cm_p_tss)
dominance_matrix_cm_p_tss = dominance_dict["dominance_matrix"]
dominance_graph_cm_p_tss = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [173]:
dominance_matrix_cm_p_tss.shape, len(dominance_graph_cm_p_tss)

((20, 20), 20)

<IPython.core.display.Javascript object>

In [174]:
df_sorted_topo["Classification"].value_counts()

Classification
intermediate_model    17
non_dominant_model     2
dominant_model         1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [175]:
df_sorted_topo.to_csv(
    "../../../../reports/results/local_models/ecics/204_ab_dominance_analysis_cm_p_tss_last_fold.csv",
    index=False,
)

<IPython.core.display.Javascript object>

In [176]:
df_sorted_topo.head(5)

,Company,Plant,Cross Validation,Features,Features_bkp,Model,Model_bkp,Model_bkp_2,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count
0,204,ab,Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Mineralogical + Physical - Early CS,Neural Networks,BidirectionalGRU,BidirectionalGRU_1.0,1.0,1.245885,0.954780,2.220923,0.944251,-5.643177,0,dominant_model,19
1,204,ab,Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Mineralogical + Physical - Early CS,Neural Networks,BidirectionalLSTM,BidirectionalLSTM_1.0,1.0,1.278895,0.983701,2.269825,0.941258,-5.330194,1,intermediate_model,18
2,204,ab,Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Mineralogical + Physical - Early CS,Neural Networks,GRU,GRU_1.0,1.0,1.311909,0.996029,2.307897,0.938186,-5.090212,2,intermediate_model,16
3,204,ab,Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Mineralogical + Physical - Early CS,Neural Networks,LSTM,LSTM_1.0,1.0,1.352942,0.991959,2.288953,0.934258,-4.958574,2,intermediate_model,16
12,204,ab,Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Mineralogical + Physical - Early CS,Neural Networks,GRU,GRU_14.0,14.0,1.520913,1.236619,2.844728,0.916465,-2.511100,4,intermediate_model,11


<IPython.core.display.Javascript object>

In [177]:
df_sorted_topo_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "RMSE_mean",
        "MAE_mean",
        "MAPE_mean",
        "R2_mean",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
].round(2)

<IPython.core.display.Javascript object>

###### Dominant Models

In [178]:
# Top 5
df_sorted_topo_cols.sort_values(
    by=["Dominates_Count", "SCPM"], ascending=[False, True]
).iloc[0:5].round(2)

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
0,BidirectionalGRU_1.0,1.25,0.95,2.22,0.94,-5.64,0,dominant_model,19,ab
1,BidirectionalLSTM_1.0,1.28,0.98,2.27,0.94,-5.33,1,intermediate_model,18,ab
2,GRU_1.0,1.31,1.00,2.31,0.94,-5.09,2,intermediate_model,16,ab
3,LSTM_1.0,1.35,0.99,2.29,0.93,-4.96,2,intermediate_model,16,ab
17,MLP,1.53,1.14,2.64,0.92,-3.13,4,intermediate_model,14,ab


<IPython.core.display.Javascript object>

In [179]:
# Top 4
df_sorted_topo_cols.iloc[0:5].sort_values(by="Dominates_Count", ascending=False)

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
0,BidirectionalGRU_1.0,1.25,0.95,2.22,0.94,-5.64,0,dominant_model,19,ab
1,BidirectionalLSTM_1.0,1.28,0.98,2.27,0.94,-5.33,1,intermediate_model,18,ab
2,GRU_1.0,1.31,1.00,2.31,0.94,-5.09,2,intermediate_model,16,ab
3,LSTM_1.0,1.35,0.99,2.29,0.93,-4.96,2,intermediate_model,16,ab
12,GRU_14.0,1.52,1.24,2.84,0.92,-2.51,4,intermediate_model,11,ab


<IPython.core.display.Javascript object>

In [180]:
df_sorted_topo_cols[df_sorted_topo_cols["Classification"].eq("dominant_model")].shape

(1, 10)

<IPython.core.display.Javascript object>

###### Intermediate models

In [181]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:3]

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
1,BidirectionalLSTM_1.0,1.28,0.98,2.27,0.94,-5.33,1,intermediate_model,18,ab
2,GRU_1.0,1.31,1.00,2.31,0.94,-5.09,2,intermediate_model,16,ab
3,LSTM_1.0,1.35,0.99,2.29,0.93,-4.96,2,intermediate_model,16,ab


<IPython.core.display.Javascript object>

In [182]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].shape

(17, 10)

<IPython.core.display.Javascript object>

###### non-dominant models

In [183]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:5]

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
8,LSTM_7.0,2.63,1.69,4.04,0.75,6.70,16,non_dominant_model,0,ab
5,BidirectionalGRU_7.0,2.51,1.85,4.28,0.77,6.92,18,non_dominant_model,0,ab


<IPython.core.display.Javascript object>

In [184]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].shape

(2, 10)

<IPython.core.display.Javascript object>

###### Statistics per model type

In [185]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "RMSE_mean",
        "MAE_mean",
        "MAPE_mean",
        "R2_mean",
        "SCPM",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [186]:
df_sorted_topo["Model"].value_counts()

Model
Neural Networks      16
Trees                 3
Linear Regression     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [187]:
df_sorted_topo_models_grouped = df_sorted_topo_models_cols.groupby("Model").describe().T

<IPython.core.display.Javascript object>

In [188]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [189]:
(
    df_sorted_topo_models_cols.groupby("Model")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model             Classification                
Linear Regression intermediate_model      100.00
Neural Networks   intermediate_model       81.25
                  non_dominant_model       12.50
                  dominant_model            6.25
Trees             intermediate_model      100.00

<IPython.core.display.Javascript object>

In [190]:
df_sorted_topo_models_cols.groupby("Model").describe().round(2).T

Model                  Linear Regression  Neural Networks  Trees
Dominated_Count count                1.0            16.00   3.00
                mean                12.0             8.12   8.67
                std                  NaN             6.13   3.21
                min                 12.0             0.00   5.00
                25%                 12.0             3.50   7.50
                50%                 12.0             6.00  10.00
                75%                 12.0            12.75  10.50
                max                 12.0            18.00  11.00
Dominates_Count count                1.0            16.00   3.00
                mean                 2.0             8.94   7.67
                std                  NaN             6.51   3.21
                min                  2.0             0.00   4.00
                25%                  2.0             3.75   6.50
                50%                  2.0            10.00   9.00
                75%                  2.0            14.50   9.50
                max                  2.0            19.00  10.00

<IPython.core.display.Javascript object>

In [191]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [192]:
df_sorted_topo_models_cols.groupby("Model")["Classification"].describe()

,count,unique,top,freq
Model,,,,
Linear Regression,1,1,intermediate_model,1
Neural Networks,16,3,intermediate_model,13
Trees,3,1,intermediate_model,3


<IPython.core.display.Javascript object>

In [193]:
df_sorted_topo_models_cols.groupby("Model")[["Classification"]].value_counts()

Model              Classification    
Linear Regression  intermediate_model     1
Neural Networks    intermediate_model    13
                   non_dominant_model     2
                   dominant_model         1
Trees              intermediate_model     3
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [194]:
summary_stats = df_sorted_topo_models_cols.groupby("Model").agg(
    Total_Models=("Model", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [195]:
summary_stats.round(2)

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model,,,,,,
Linear Regression,1,2.00,2.0,12,2,0.10
Neural Networks,16,8.94,10.0,130,143,7.15
Trees,3,7.67,9.0,26,23,1.15


<IPython.core.display.Javascript object>

In [196]:
df_sorted_topo_models_grouped.T

RMSE_mean                                                    \
                      count      mean       std       min       25%       50%   
Model                                                                           
Linear Regression       1.0  2.020479       NaN  2.020479  2.020479  2.020479   
Neural Networks        16.0  1.799239  0.448305  1.245885  1.478920  1.669965   
Trees                   3.0  1.961140  0.190604  1.807186  1.854546  1.901907   

                                      MAE_mean            ... Dominated_Count  \
                        75%       max    count      mean  ...             75%   
Model                                                     ...                   
Linear Regression  2.020479  2.020479      1.0  1.696536  ...           12.00   
Neural Networks    2.078609  2.625634     16.0  1.357469  ...           12.75   
Trees              2.038118  2.174328      3.0  1.287201  ...           10.50   

                        Dominates_Count                                       \
                    max           count      mean       std  min   25%   50%   
Model                                                                          
Linear Regression  12.0             1.0  2.000000       NaN  2.0  2.00   2.0   
Neural Networks    18.0            16.0  8.937500  6.506087  0.0  3.75  10.0   
Trees              11.0             3.0  7.666667  3.214550  4.0  6.50   9.0   

                               
                    75%   max  
Model                          
Linear Regression   2.0   2.0  
Neural Networks    14.5  19.0  
Trees               9.5  10.0  

[3 rows x 56 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [197]:
cols = [
    # "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "RMSE_mean",
    "MAE_mean",
    "MAPE_mean",
    "R2_mean",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

#### SCPM Analysis by Project

In [198]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [199]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [200]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(5)

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
0,BidirectionalGRU_1.0,1.25,0.95,2.22,0.94,-5.64,0,19,dominant_model,ab,True
1,BidirectionalLSTM_1.0,1.28,0.98,2.27,0.94,-5.33,1,18,intermediate_model,ab,True
2,GRU_1.0,1.31,1.00,2.31,0.94,-5.09,2,16,intermediate_model,ab,True
3,LSTM_1.0,1.35,0.99,2.29,0.93,-4.96,2,16,intermediate_model,ab,True
17,MLP,1.53,1.14,2.64,0.92,-3.13,4,14,intermediate_model,ab,False


<IPython.core.display.Javascript object>

### Variable Grouping: CM-P-CS

In [201]:
df_results["Features"].unique()

array(['Chemical + Mineralogical', 'Chemical + Mineralogical + Physical',
       'Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [202]:
df_results_cm_p_cs = (
    df_results[df_results["Features"].eq("Chemical + Mineralogical + Physical")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [203]:
df_results_cm_p_cs["Features"].unique()

array(['Chemical + Mineralogical + Physical'], dtype=object)

<IPython.core.display.Javascript object>

In [204]:
df_results_cm_p.shape

(20, 13)

<IPython.core.display.Javascript object>

### Time Series Split

In [205]:
df_results_cm_p_cs["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [206]:
df_results_cm_p_cs_tss = (
    df_results_cm_p_cs[df_results_cm_p_cs["Cross Validation"].eq("Time Series Split")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [207]:
df_results_cm_p_cs_tss["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [208]:
df_results_cm_p_cs_tss.shape

(20, 13)

<IPython.core.display.Javascript object>

###### SCPM computation

In [209]:
df_results_cm_p_cs_tss = compute_scpm(df_results_cm_p_cs_tss)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [210]:
# Per project
dominance_dict = make_dominance_analysis(df_results_cm_p_cs_tss)
dominance_matrix_cm_p_cs_tss = dominance_dict["dominance_matrix"]
dominance_graph_cm_p_cs_tss = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [211]:
dominance_matrix_cm_p_cs_tss.shape, len(dominance_graph_cm_p_cs_tss)

((20, 20), 20)

<IPython.core.display.Javascript object>

In [212]:
df_sorted_topo["Classification"].value_counts()

Classification
intermediate_model    17
dominant_model         2
non_dominant_model     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [213]:
df_sorted_topo.to_csv(
    "../../../../reports/results/local_models/ecics/204_ab_dominance_analysis_cm_p_cs_tss_last_fold.csv",
    index=False,
)

<IPython.core.display.Javascript object>

In [214]:
df_sorted_topo.head(5)

,Company,Plant,Cross Validation,Features,Features_bkp,Model,Model_bkp,Model_bkp_2,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count
4,204,ab,Time Series Split,Chemical + Mineralogical + Physical,Chemical + Mineralogical + Physical,Neural Networks,Transformer,Transformer_1.0,1.0,1.011624,0.777037,1.800450,0.963245,-5.971604,0,dominant_model,17
16,204,ab,Time Series Split,Chemical + Mineralogical + Physical,Chemical + Mineralogical + Physical,Linear Regression,Linear Regression,Linear Regression,NaN,1.016610,0.733020,1.732475,0.962882,-6.438778,0,dominant_model,18
18,204,ab,Time Series Split,Chemical + Mineralogical + Physical,Chemical + Mineralogical + Physical,Trees,Random Forest,Random Forest,NaN,1.020357,0.746749,1.752824,0.962607,-6.237643,1,intermediate_model,17
15,204,ab,Time Series Split,Chemical + Mineralogical + Physical,Chemical + Mineralogical + Physical,Trees,Decision Tree,Decision Tree,NaN,1.122872,0.826072,1.926232,0.954716,-3.982846,3,intermediate_model,16
5,204,ab,Time Series Split,Chemical + Mineralogical + Physical,Chemical + Mineralogical + Physical,Neural Networks,BidirectionalGRU,BidirectionalGRU_7.0,7.0,1.138369,0.863000,2.021652,0.953260,-3.230350,4,intermediate_model,14


<IPython.core.display.Javascript object>

In [215]:
df_sorted_topo_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "RMSE_mean",
        "MAE_mean",
        "MAPE_mean",
        "R2_mean",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
].round(2)

<IPython.core.display.Javascript object>

###### Dominant Models

In [216]:
# Top 5
df_sorted_topo_cols.sort_values(
    by=["Dominates_Count", "SCPM"], ascending=[False, True]
).iloc[0:5].round(2)

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
16,Linear Regression,1.02,0.73,1.73,0.96,-6.44,0,dominant_model,18,ab
18,Random Forest,1.02,0.75,1.75,0.96,-6.24,1,intermediate_model,17,ab
4,Transformer_1.0,1.01,0.78,1.80,0.96,-5.97,0,dominant_model,17,ab
15,Decision Tree,1.12,0.83,1.93,0.95,-3.98,3,intermediate_model,16,ab
5,BidirectionalGRU_7.0,1.14,0.86,2.02,0.95,-3.23,4,intermediate_model,14,ab


<IPython.core.display.Javascript object>

In [217]:
df_sorted_topo_cols[df_sorted_topo_cols["Classification"].eq("dominant_model")].shape

(2, 10)

<IPython.core.display.Javascript object>

###### Intermediate models

In [218]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:4]

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
18,Random Forest,1.02,0.75,1.75,0.96,-6.24,1,intermediate_model,17,ab
15,Decision Tree,1.12,0.83,1.93,0.95,-3.98,3,intermediate_model,16,ab
5,BidirectionalGRU_7.0,1.14,0.86,2.02,0.95,-3.23,4,intermediate_model,14,ab
19,XGBoost,1.14,0.87,2.08,0.95,-2.98,4,intermediate_model,14,ab


<IPython.core.display.Javascript object>

In [219]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].shape

(17, 10)

<IPython.core.display.Javascript object>

###### non-dominant models

In [220]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by=["Dominated_Count", "SCPM"], ascending=[True, False]).iloc[:5]

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
13,LSTM_14.0,1.56,1.29,2.89,0.91,7.83,19,non_dominant_model,0,ab


<IPython.core.display.Javascript object>

In [221]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].shape

(1, 10)

<IPython.core.display.Javascript object>

###### Statistics per model type

In [222]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "RMSE_mean",
        "MAE_mean",
        "MAPE_mean",
        "R2_mean",
        "SCPM",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [223]:
df_sorted_topo["Model"].value_counts()

Model
Neural Networks      16
Trees                 3
Linear Regression     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [224]:
df_sorted_topo_models_grouped = df_sorted_topo_models_cols.groupby("Model").describe().T

<IPython.core.display.Javascript object>

In [225]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [226]:
(
    df_sorted_topo_models_cols.groupby("Model")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model             Classification                
Linear Regression dominant_model          100.00
Neural Networks   intermediate_model       87.50
                  dominant_model            6.25
                  non_dominant_model        6.25
Trees             intermediate_model      100.00

<IPython.core.display.Javascript object>

In [227]:
df_sorted_topo_models_cols.groupby("Model").describe().round(2).T

Model                  Linear Regression  Neural Networks  Trees
Dominated_Count count                1.0            16.00   3.00
                mean                 0.0            10.81   2.67
                std                  NaN             5.18   1.53
                min                  0.0             0.00   1.00
                25%                  0.0             7.50   2.00
                50%                  0.0            11.00   3.00
                75%                  0.0            15.00   3.50
                max                  0.0            19.00   4.00
Dominates_Count count                1.0            16.00   3.00
                mean                18.0             7.25  15.67
                std                  NaN             5.13   1.53
                min                 18.0             0.00  14.00
                25%                 18.0             2.00  15.00
                50%                 18.0             7.00  16.00
                75%                 18.0            11.25  16.50
                max                 18.0            17.00  17.00

<IPython.core.display.Javascript object>

In [228]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [229]:
df_sorted_topo_models_cols.groupby("Model")["Classification"].describe()

,count,unique,top,freq
Model,,,,
Linear Regression,1,1,dominant_model,1
Neural Networks,16,3,intermediate_model,14
Trees,3,1,intermediate_model,3


<IPython.core.display.Javascript object>

In [230]:
df_sorted_topo_models_cols.groupby("Model")[["Classification"]].value_counts()

Model              Classification    
Linear Regression  dominant_model         1
Neural Networks    intermediate_model    14
                   dominant_model         1
                   non_dominant_model     1
Trees              intermediate_model     3
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [231]:
summary_stats = df_sorted_topo_models_cols.groupby("Model").agg(
    Total_Models=("Model", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [232]:
summary_stats.round(2)

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model,,,,,,
Linear Regression,1,18.00,18.0,0,18,0.90
Neural Networks,16,7.25,7.0,173,116,5.80
Trees,3,15.67,16.0,8,47,2.35


<IPython.core.display.Javascript object>

In [233]:
df_sorted_topo_models_grouped.T

RMSE_mean                                                    \
                      count      mean       std       min       25%       50%   
Model                                                                           
Linear Regression       1.0  1.016610       NaN  1.016610  1.016610  1.016610   
Neural Networks        16.0  1.323145  0.146398  1.011624  1.238889  1.300713   
Trees                   3.0  1.094057  0.064329  1.020357  1.071615  1.122872   

                                      MAE_mean            ... Dominated_Count  \
                        75%       max    count      mean  ...             75%   
Model                                                     ...                   
Linear Regression  1.016610  1.016610      1.0  0.733020  ...             0.0   
Neural Networks    1.436520  1.561301     16.0  1.024943  ...            15.0   
Trees              1.130906  1.138941      3.0  0.815470  ...             3.5   

                        Dominates_Count                                   \
                    max           count       mean       std   min   25%   
Model                                                                      
Linear Regression   0.0             1.0  18.000000       NaN  18.0  18.0   
Neural Networks    19.0            16.0   7.250000  5.131601   0.0   2.0   
Trees               4.0             3.0  15.666667  1.527525  14.0  15.0   

                                      
                    50%    75%   max  
Model                                 
Linear Regression  18.0  18.00  18.0  
Neural Networks     7.0  11.25  17.0  
Trees              16.0  16.50  17.0  

[3 rows x 56 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [234]:
cols = [
    # "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "RMSE_mean",
    "MAE_mean",
    "MAPE_mean",
    "R2_mean",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

#### SCPM Analysis by Project

In [235]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [236]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [237]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(5)

,Model_bkp_2,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
16,Linear Regression,1.02,0.73,1.73,0.96,-6.44,0,18,dominant_model,ab,False
18,Random Forest,1.02,0.75,1.75,0.96,-6.24,1,17,intermediate_model,ab,False
4,Transformer_1.0,1.01,0.78,1.80,0.96,-5.97,0,17,dominant_model,ab,False
15,Decision Tree,1.12,0.83,1.93,0.95,-3.98,3,16,intermediate_model,ab,True
5,BidirectionalGRU_7.0,1.14,0.86,2.02,0.95,-3.23,4,14,intermediate_model,ab,True


<IPython.core.display.Javascript object>